In [1]:
import numpy as np
import pandas as pd
import pickle
import re
import os
import flask
import json
from scipy.misc import imread
from keras.models import load_model

Using TensorFlow backend.


In [2]:
rootpath = "../../egypt/EgyptianHieroglyphDataset/MyTrainTest"
original_dim = (75, 50)
batch_size = 32 # need to be exactly the same as training

In [3]:
clf = load_model("../models/clf.h5")

In [4]:
try:
    with open("../models/dict.pkl", "rb") as pfile:
        symbol_df = pickle.load(pfile)
except:
    tr_rootpath = os.path.join(rootpath, "train")
    symbol_df = [x for x in os.listdir(tr_rootpath) if re.search(r"UNKNOWN|^\.", x) == None]
    symbol_df = pd.DataFrame(sorted(symbol_df))
    symbol_df.reset_index(inplace = True)
    symbol_df.columns = ["symbol_num", "symbol"]
    with open("../models/dict.pkl", "wb") as pfile:
        pickle.dump(symbol_df, pfile)
# symbol_df.tail(1)

In [5]:
# arbitrarily pick one file
tr_rootpath = os.path.join(rootpath, "train")
subdir = os.path.join(tr_rootpath, 'O4')
pngfile = os.path.join(subdir, '220172_O4.png')

### get img

In [6]:
arrayfile = imread(pngfile, flatten = True, mode = "RGB").astype('float32')

In [ ]:
arrayfile = np.array([arrayfile for i in range(batch_size)])
arrayfile = np.reshape(arrayfile, (batch_size, 1) + (original_dim)) / 255.

### pred

In [ ]:
pred_array = clf.predict(arrayfile, batch_size = batch_size)

In [ ]:
pred_cat = np.argmax(pred_array, 1)[0]

In [ ]:
cond = symbol_df["symbol_num"] == pred_cat
pred = list(symbol_df.loc[cond, "symbol"])[0]
pred